In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


## Audio transcription
@forusone (shins777@gmail.com)

### Transcribing audio file
This colab explains how to transcribe audio file with speech api(STT)

### Install Vertex AI SDK for Python

In [2]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    google-cloud-speech \
    google-cloud-storage \
    ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 62.8 MB/s eta 0:00:00


### GCP Authentication

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik


Updated property [core/project].


### Initial set up

In [4]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-pro-002"

import base64
import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)
model = GenerativeModel(MODEL)

### Helper functions



In [5]:

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket.

    """

    from google.cloud import storage

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

def transcribe_audio(audio_uri:str):
  """Transcribe an audio file."""

  from google.cloud import speech

  # Instantiates a client
  client = speech.SpeechClient()
  audio = speech.RecognitionAudio(uri=audio_uri)

  config = speech.RecognitionConfig(
      #encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
      encoding=speech.RecognitionConfig.AudioEncoding.MP3,
      sample_rate_hertz=16000,
      language_code="en-US",
      model="video",  # Chosen model
  )

  operation = client.long_running_recognize(config=config, audio=audio)

  print("Waiting for operation to complete...")
  response = operation.result(timeout=90)  # Set a timeout

  # Process the response
  for i, result in enumerate(response.results):
      alternative = result.alternatives[0]
      print("-" * 20)
      print(f"First alternative of result {i}")
      print(f"Transcript: {alternative.transcript}")

### export audio file from video file.

In [1]:
!ffmpeg -i './Stephen Hawking Discovers.mp4' './Stephen Hawking Discovers.mp3'

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### Upload audio(mp3) file.

In [7]:
bucket_name = "tests_nov25_2024"
source_file_name = "Stephen Hawking Discovers.mp3"
destination_blob_name = f"translation/{source_file_name}"

upload_blob(bucket_name, source_file_name, destination_blob_name)

File Stephen Hawking Discovers.mp3 uploaded to translation/Stephen Hawking Discovers.mp3.


### Transcribe audio file.

In [10]:
audio_uri = f"gs://{bucket_name}/{destination_blob_name}"
transcribe_audio(audio_uri )

Waiting for operation to complete...
--------------------
First alternative of result 0
Transcript: start
--------------------
First alternative of result 1
Transcript:  more than three times the size of our sun or to end its life how
--------------------
First alternative of result 2
Transcript:  with a collapse the gravitational forces of the entire Mass overcoming the electromagnetic forces of individual atoms and so collapsing inwards if the star is massive enough it will continue this collapse creating a black hole where the warping of space-time is so great that nothing can escape not even light it gets
--------------------
First alternative of result 3
Transcript:  smaller
--------------------
First alternative of result 4
Transcript:  smaller the star in fact gets denser as atoms even subatomic particles get literally crushed into smaller and smaller space
--------------------
First alternative of result 5
Transcript:  it's endpoint what are we left with
--------------------
Fi